## 6.2 k分割交差検証を使ったモデル性能評価

- 訓練データに対してモデルが単純すぎると学習不足、複雑すぎると過学習に陥る
  - モデルにとって未知のデータを使用してモデル評価を行う必要がある
- 代表的なモデルの評価方法
  - ホールドアウト法
  - k分割交差検証

### 6.2.1 ホールドアウト法

データを訓練データ、検証データ、テストデータに分割する
- メリット
  - 未知のデータに対するモデルの汎化能力のバイアスが低くなる
- デメリット
  - データの抽出方法によって評価が変わる
  - 訓練データセットと検証データセットをどのように分割するかによって、性能の評価に影響が及ぶ

1. もともとのデータを訓練データセット・テストデータセットに分ける
2. 訓練データセットを訓練データセット・検証データセット（8:2）
  - 検証データセットはハイパーパラメータの調整に使われる
  - テストデータセットは最後の評価に行われる

### 6.2.2 k分割交差検証法
ホールドアウト法を改良した手法（ホールドアウト法をk回繰り返す）

- メリット
  - バリアンスが低い結果が得られる
- デメリット
  - 学習回数が増えるため、実行時間が長くなる

1. 非復元抽出を用いて、訓練データセットをランダムにk個に分割して、k-1個を訓練に使い、1個を評価に使う
2. モデルの訓練と評価をk回繰り返す
3. k個のモデルの平均性能を計算

#### 層化k分割交差検証

各サブセットのクラスラベルの比率が維持されたまま分割される  
→評価のバイアスとバリアンスが改善される





In [3]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

data = load_breast_cancer()

X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

In [17]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

pipe_lr = make_pipeline(
    StandardScaler(),
    PCA(n_components=10),
    LogisticRegression(penalty='l2', random_state=10, solver='lbfgs')
)

In [18]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=10).split(X_train, y_train)

scores = []

for k, (train, test) in enumerate(kfold):
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    scores.append(score)
    print('Fold: {:2d}, Class dist.: {}, Acc: {:.3f}'.format(k+1, np.bincount(y_train[train]), score))

print('\nCV accuracy: {:.3f} +/- {:.3f}'.format(np.mean(scores), np.std(scores)))

Fold:  1, Class dist.: [153 256], Acc: 0.978
Fold:  2, Class dist.: [153 256], Acc: 1.000
Fold:  3, Class dist.: [153 256], Acc: 0.957
Fold:  4, Class dist.: [153 256], Acc: 0.978
Fold:  5, Class dist.: [153 256], Acc: 0.978
Fold:  6, Class dist.: [153 257], Acc: 1.000
Fold:  7, Class dist.: [153 257], Acc: 0.978
Fold:  8, Class dist.: [153 257], Acc: 0.978
Fold:  9, Class dist.: [153 257], Acc: 0.933
Fold: 10, Class dist.: [153 257], Acc: 1.000

CV accuracy: 0.978 +/- 0.020


In [26]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(estimator=pipe_lr, X=X_train, y=y_train, cv=10, n_jobs=1)

print('CV accuracy scores:')
for k, score in enumerate(scores):
    print(' - {}: {}'.format(k+1, score))
    
print('\nCV accuracy: {:.3f} +/- {:.3f}'.format(np.mean(scores), np.std(scores)))

CV accuracy scores:
 - 1: 0.9782608695652174
 - 2: 1.0
 - 3: 0.9565217391304348
 - 4: 0.9782608695652174
 - 5: 0.9782608695652174
 - 6: 1.0
 - 7: 0.9777777777777777
 - 8: 0.9777777777777777
 - 9: 0.9333333333333333
 - 10: 1.0

CV accuracy: 0.978 +/- 0.020
